In [0]:
#connecting to storage
storageAccountName = "e2eazflow"
storageAccountAccessKey = "BUBBj+jJtgmz7PAOVXSJdphoL527DoL8EsxBWUTAtLHeQq9qYbHae2L6y5+W14vjfdg8TIBru7fO+AStzcQaYw=="
blobContainerName = "e2e-data"
mountPoint = "/mnt/data/"
if not any(mount.mountPoint == mountPoint for mount in dbutils.fs.mounts()):
  try:
    dbutils.fs.mount(
      source = "wasbs://{}@{}.blob.core.windows.net".format(blobContainerName, storageAccountName),
      mount_point = mountPoint,
      extra_configs = {'fs.azure.account.key.' + storageAccountName + '.blob.core.windows.net': storageAccountAccessKey}
    #   extra_configs = {'fs.azure.sas.' + blobContainerName + '.' + storageAccountName + '.blob.core.windows.net': sasToken}
    )
    print("mount succeeded!")
  except Exception as e:
    print("mount exception", e)

In [0]:
%fs 
ls "mnt/data"

path,name,size,modificationTime
dbfs:/mnt/data/raw/,raw/,0,0
dbfs:/mnt/data/transformed/,transformed/,0,0


In [0]:
#test load data
artists = spark.read.format('csv').option("header","true").option("inferSchema","true").load("/mnt/data/raw/artists.csv")
artists.show()

+--------+-----------------+
|reviewid|           artist|
+--------+-----------------+
|   22703|   massive attack|
|   22721|         krallice|
|   22659|     uranium club|
|   22661|          kleenex|
|   22661|          liliput|
|   22725|             taso|
|   22722|  various artists|
|   22704|      little simz|
|   22694|       yotam avni|
|   22714|        brian eno|
|   22724|          the lox|
|   22715|    harry bertoia|
|   22745|   run the jewels|
|   22700|   steven warwick|
|   22720|       yasiin bey|
|   22699|        smoke dza|
|   22699|        pete rock|
|   22665|  various artists|
|   22666|          senyawa|
|   22719|chance the rapper|
+--------+-----------------+
only showing top 20 rows



In [0]:
#loading the rest (lazy evaluation)
content = spark.read.format('csv').option("header","true").option("inferSchema","true").load("/mnt/data/raw/content.csv")
genres = spark.read.format('csv').option("header","true").option("inferSchema","true").load("/mnt/data/raw/genres.csv")
labels = spark.read.format('csv').option("header","true").option("inferSchema","true").load("/mnt/data/raw/labels.csv")
reviews = spark.read.format('csv').option("header","true").option("inferSchema","true").load("/mnt/data/raw/reviews.csv")
years = spark.read.format('csv').option("header","true").option("inferSchema","true").load("/mnt/data/raw/years.csv")

In [0]:
#start wrting the transformed spark data to storage
artists.write.option("header","true").csv("/mnt/data/transformed/artists.csv")

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-3092386258597600>:2
      1 #start wrting the transformed spark data to storage
----> 2 artists.write.option("header","true").csv("/mnt/data/transformed/artists.csv")

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:1798, in DataFrameWriter.csv(self, path, mode, compression, sep, quote, escape, header, nullValue, escapeQuotes, quoteAll, dateFormat, timestampFormat, ignoreLeadingWhiteSpace, ignoreTrailingWhiteSpace, charToEscapeQuoteEscaping, encodin

In [0]:
content.write.option("header","true").csv("/mnt/data/transformed/content.csv")
genres.write.option("header","true").csv("/mnt/data/transformed/genres.csv")
labels.write.option("header","true").csv("/mnt/data/transformed/labels.csv")
reviews.write.option("header","true").csv("/mnt/data/transformed/reviews.csv")
years.write.option("header","true").csv("/mnt/data/transformed/years.csv")